In [1]:
# unset PYTHONPATH first
from ko_text import *
from ko_crawler import *

In [2]:
nlp = NLP()

# morphs

In [3]:
train_df = pd.read_csv('Data/morphs/train_morphs_final.csv', encoding = 'utf-8')
test_df = pd.read_csv('Data/morphs/test_morphs_final.csv', encoding = 'utf-8')

# 용량을 줄이기 위해 '단어 단어' 꼴로 묶어둔 token을 ['단어', '단어'] 꼴로 풀기
train_df['Token'] = [token.split() for token in train_df['Token']]
test_df['Token'] = [token.split() for token in test_df['Token']]

# nouns

In [4]:
'''train_df = pd.read_csv('Data/nouns/train_nouns_final.csv', encoding = 'utf-8')
test_df = pd.read_csv('Data/nouns/test_nouns_final.csv', encoding = 'utf-8')

# 용량을 줄이기 위해 '단어 단어' 꼴로 묶어둔 token을 ['단어', '단어'] 꼴로 풀기
train_df['Token'] = [token.split() for token in train_df['Token']]
test_df['Token'] = [token.split() for token in test_df['Token']]'''

In [4]:
print(train_df.shape)

(82963, 3)


In [5]:
Counter(train_df['Section'])

Counter({'IT과학': 9836,
         '경제': 8050,
         '국제': 9677,
         '기업': 9604,
         '문화': 9313,
         '부동산': 9714,
         '사회': 9325,
         '정치': 8875,
         '증권': 8569})

In [6]:
train_df.head()

,Token,Section,Num of Tokens
0,"[농협, 금융, 지주, 김광수, 회장, 서대문, 본사, 에서, 농협, 금융, 자회사...",경제,103
1,"[코스피, 붕괴하면서, 달러, 환율, 으로, 뛰었다, 서울, 외환시장, 에서, 달러...",경제,218
2,"[정의당, 국회, 롯데, 갑질, 피해자, 간담, 개최, 김상조, 정거, 위원장, 공...",경제,226
3,"[코스피, 무역, 전쟁, 내외, 악재, 힘없이, 내줬다, 코스피, 거래, 일보, 포...",경제,172
4,"[제주항공, 호텔, 개점, 분석, 고객, 자유, 여행객, 제주항공, 올해, 서울, ...",경제,169


In [7]:
# 한 label마다 학습할 단어의 수
train_size_for_each_label = 1000
test_size_for_each_label = 300

n_class = len(train_df['Section'].unique())


# Doc2Vec으로 vector를 추정하기 위한 split 과정
train_batch_size = n_class * train_size_for_each_label
test_batch_size = n_class * test_size_for_each_label

train_token_ls_split, train_tag_ls_split = nlp.extract_a_equally_splited_batch(train_df['Token'], train_df['Section'], train_batch_size)
test_token_ls_split, test_tag_ls_split =  nlp.extract_a_equally_splited_batch(test_df['Token'],test_df['Section'], test_batch_size)

<br>


<br>


# 잘 뽑혔는지 확인

In [8]:
' '.join(train_token_ls_split[0])

'증권 디스플레이 대해 중소 유기발광다이오드 실적 불확실 존재 한다며 목표 주가 에서 으로 내린다고 밝혔다 김동원 연구원 올해 부터 디스플레이 플라스틱 패널 신규 공급 시작 하기 위해 파주 공장 에서 중소 신규 라인 가동 이라고 예상 했다 이어 이는 내년 하반기 북미 전략 고객 신제품 수요 대응 하고 부터 시장 확대 예상 되는 더블 스마트폰 시장 전략 으로 대응 하기 으로 판단 된다고 덧붙였다 연구원 다만 향후 중소 라인 개선 강도 연간 규모 감가상각비 으로 실적 개선 불확실 상존 한다고 설명 했다'

In [9]:
train_tag_ls_split[0]

'기업'

# Doc2Vec Parameter 튜닝

## **set ALPHA as default**

In [11]:
%%time

result_dict = {
                'corpus_count' : [],
               'min_count' : [],
               'vector_size' : [],
               'window' : [],
               'n_epochs' : [],
               'accuracy' : [],
               'sample' : [],
               'dm' : [],
              }

testing_section_ls = ['경제','기업','사회','국제','부동산','증권','정치','IT과학','문화']

# 하이퍼 파라미터 튜닝 작업 수행

# 언더샘플링으로 학습
undersampling_size = Counter(train_df['Section']).most_common()[-1][1]

for dm in [1, 0]:
    for size in [10000 * 9]:
        x_split, y_split = nlp.extract_a_equally_splited_batch(train_df['Token'].tolist(), train_df['Section'].tolist(), size)
        for sample in [1e-04, 1e-05, 1e-06]:
            for min_count in [10, 50, 100]:
                for vector_size in [100,300]:
                    for window in [5,15]:
                        for n_epochs in [10]:

                            # Doc2Vec 모델 생성
                            nlp.make_Doc2Vec_model(dm = dm,
                                                   min_count = min_count,
                                                   sample = sample,
                                                   vector_size = vector_size,
                                                   window = window,
                                                   dm_mean = 0,
                                                   dm_concat = 0)

                            nlp.build_and_train_Doc2Vec_model(x_split,
                                                              y_split,
                                                              n_epochs = n_epochs)


                            model_name = 'Doc2Vec_dm=%s&cc=%s&vs=%s&win=%s&min=%s&sample=%s&epochs=%s'%(\
                                                                                                                 nlp.Doc2Vec_model.dm,
                                                                                                                   nlp.Doc2Vec_model.corpus_count,
                                                                                                                   nlp.Doc2Vec_model.vector_size,
                                                                                                                   nlp.Doc2Vec_model.window,
                                                                                                                   nlp.Doc2Vec_model.min_count,
                                                                                                                   nlp.Doc2Vec_model.sample,
                                                                                                                   nlp.Doc2Vec_model.epochs)
                            # Doc2Vec 모델 저장
                            #nlp.Doc2Vec_model.save('Doc2Vec_model/'+model_name)


                            '''

                            X =nlp.infer_vectors_with_Doc2Vec(train_token_for_display[0])

                            from sklearn.decomposition import PCA

                            pca = PCA(n_components=2)
                            X_pca = pca.fit_transform(X)
                            scatter_df = pd.DataFrame(X_pca,
                                                      index = train_tag_for_display[0],
                                                      columns = ['x','y'])

                            plt.figure(figsize = (15, 15))

                            for section in testing_section_ls:
                                temp_df = scatter_df[scatter_df.index == section]
                                plt.scatter(temp_df['x'].values, temp_df['y'].values, label = section, c = np.random.rand(3,))

                            plt.legend(loc = 'best')
                            plt.savefig('Doc2Vec_model/images/'+model_name)

                            '''
                            # 각 레이블별 1000개씩 학습, 

                            # LR
                            X_train = nlp.infer_vectors_with_Doc2Vec(train_token_ls_split)
                            y_train = train_tag_ls_split

                            X_test = nlp.infer_vectors_with_Doc2Vec(test_token_ls_split)
                            y_test = test_tag_ls_split


                            clf = LogisticRegression(solver = 'sag',
                                                     multi_class = 'multinomial')


                            clf.fit(X_train, y_train)
                            y_pred = clf.predict(X_test)
                            
                            result_dict['dm'].append(nlp.Doc2Vec_model.dm)
                            result_dict['corpus_count'].append(nlp.Doc2Vec_model.corpus_count)
                            result_dict['min_count'].append(nlp.Doc2Vec_model.min_count)
                            result_dict['vector_size'].append(nlp.Doc2Vec_model.vector_size)
                            result_dict['window'].append(nlp.Doc2Vec_model.window)
                            result_dict['n_epochs'].append(nlp.Doc2Vec_model.epochs)
                            result_dict['sample'].append(nlp.Doc2Vec_model.sample)
                            result_dict['accuracy'].append(accuracy_score(y_pred, y_test))

                            print(accuracy_score(y_pred, y_test))


            pd.DataFrame(result_dict).to_csv('Parameter_tuning_result.csv', index = False)

2018-11-18 09:11:26,235 : INFO : collecting all words and their counts
2018-11-18 09:11:26,236 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2018-11-18 09:11:26,539 : INFO : PROGRESS: at example #10000, processed 2031545 words (6722740/s), 59789 word types, 2 tags
2018-11-18 09:11:26,806 : INFO : PROGRESS: at example #20000, processed 3989906 words (7348716/s), 72782 word types, 3 tags
2018-11-18 09:11:27,143 : INFO : PROGRESS: at example #30000, processed 6207278 words (6589591/s), 95998 word types, 4 tags
2018-11-18 09:11:27,451 : INFO : PROGRESS: at example #40000, processed 8204713 words (6505575/s), 111721 word types, 5 tags
2018-11-18 09:11:27,799 : INFO : PROGRESS: at example #50000, processed 10549693 words (6741093/s), 124579 word types, 6 tags
2018-11-18 09:11:28,186 : INFO : PROGRESS: at example #60000, processed 12892471 words (6067347/s), 147468 word types, 7 tags
2018-11-18 09:11:28,591 : INFO : PROGRESS: at example #70000, processed 1521

0.6788888888888889


2018-11-18 09:15:47,222 : INFO : collecting all words and their counts
2018-11-18 09:15:47,223 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2018-11-18 09:15:47,499 : INFO : PROGRESS: at example #10000, processed 2031545 words (7386538/s), 59789 word types, 2 tags
2018-11-18 09:15:47,749 : INFO : PROGRESS: at example #20000, processed 3989906 words (7850686/s), 72782 word types, 3 tags
2018-11-18 09:15:48,067 : INFO : PROGRESS: at example #30000, processed 6207278 words (6973659/s), 95998 word types, 4 tags
2018-11-18 09:15:48,368 : INFO : PROGRESS: at example #40000, processed 8204713 words (6662274/s), 111721 word types, 5 tags
2018-11-18 09:15:48,698 : INFO : PROGRESS: at example #50000, processed 10549693 words (7108328/s), 124579 word types, 6 tags
2018-11-18 09:15:49,060 : INFO : PROGRESS: at example #60000, processed 12892471 words (6488544/s), 147468 word types, 7 tags
2018-11-18 09:15:49,443 : INFO : PROGRESS: at example #70000, processed 1521

0.6844444444444444


2018-11-18 09:19:59,408 : INFO : collecting all words and their counts
2018-11-18 09:19:59,409 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2018-11-18 09:19:59,685 : INFO : PROGRESS: at example #10000, processed 2031545 words (7361273/s), 59789 word types, 2 tags
2018-11-18 09:19:59,931 : INFO : PROGRESS: at example #20000, processed 3989906 words (7981933/s), 72782 word types, 3 tags
2018-11-18 09:20:00,265 : INFO : PROGRESS: at example #30000, processed 6207278 words (6647700/s), 95998 word types, 4 tags
2018-11-18 09:20:00,566 : INFO : PROGRESS: at example #40000, processed 8204713 words (6661308/s), 111721 word types, 5 tags
2018-11-18 09:20:00,905 : INFO : PROGRESS: at example #50000, processed 10549693 words (6933053/s), 124579 word types, 6 tags
2018-11-18 09:20:01,273 : INFO : PROGRESS: at example #60000, processed 12892471 words (6377660/s), 147468 word types, 7 tags
2018-11-18 09:20:01,663 : INFO : PROGRESS: at example #70000, processed 1521

...

0.6503703703703704


2018-11-18 13:07:40,526 : INFO : collecting all words and their counts
2018-11-18 13:07:40,527 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2018-11-18 13:07:40,802 : INFO : PROGRESS: at example #10000, processed 2031545 words (7393888/s), 59789 word types, 2 tags
2018-11-18 13:07:41,052 : INFO : PROGRESS: at example #20000, processed 3989906 words (7879700/s), 72782 word types, 3 tags
2018-11-18 13:07:41,369 : INFO : PROGRESS: at example #30000, processed 6207278 words (7003370/s), 95998 word types, 4 tags
2018-11-18 13:07:41,668 : INFO : PROGRESS: at example #40000, processed 8204713 words (6697122/s), 111721 word types, 5 tags
2018-11-18 13:07:42,005 : INFO : PROGRESS: at example #50000, processed 10549693 words (6962701/s), 124579 word types, 6 tags
2018-11-18 13:07:42,374 : INFO : PROGRESS: at example #60000, processed 12892471 words (6370135/s), 147468 word types, 7 tags
2018-11-18 13:07:42,773 : INFO : PROGRESS: at example #70000, processed 1521

0.6503703703703704


2018-11-18 13:09:38,762 : INFO : collecting all words and their counts
2018-11-18 13:09:38,762 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2018-11-18 13:09:39,043 : INFO : PROGRESS: at example #10000, processed 2031545 words (7254506/s), 59789 word types, 2 tags
2018-11-18 13:09:39,293 : INFO : PROGRESS: at example #20000, processed 3989906 words (7847469/s), 72782 word types, 3 tags
2018-11-18 13:09:39,612 : INFO : PROGRESS: at example #30000, processed 6207278 words (6967482/s), 95998 word types, 4 tags
2018-11-18 13:09:39,911 : INFO : PROGRESS: at example #40000, processed 8204713 words (6682670/s), 111721 word types, 5 tags
2018-11-18 13:09:40,247 : INFO : PROGRESS: at example #50000, processed 10549693 words (7002763/s), 124579 word types, 6 tags
2018-11-18 13:09:40,610 : INFO : PROGRESS: at example #60000, processed 12892471 words (6464859/s), 147468 word types, 7 tags
2018-11-18 13:09:41,001 : INFO : PROGRESS: at example #70000, processed 1521

0.6492592592592593
CPU times: user 8h 40min 41s, sys: 19min 10s, total: 8h 59min 51s
Wall time: 4h 12s


In [13]:
nlp.Doc2Vec_model.docvecs.vectors_docs.shape

(9, 300)

<br>
